In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import graphviz

# Importation des données
data = pd.read_csv("Fraud_Data.csv")
#Visualisation du dataframe
data.head(10)

,purchase_value,source,browser,sex,age,class
0,304,SEO,Chrome,M,39,0
1,16,Ads,Chrome,F,53,0
2,15,SEO,Opera,M,53,0
3,42,Ads,Chrome,M,18,1
4,0,Ads,Chrome,F,19,0
5,27,Ads,Opera,M,34,0
6,47,SEO,Chrome,M,28,0
7,417,SEO,Chrome,F,24,0
8,54,SEO,Chrome,F,28,0
9,48,SEO,Chrome,F,27,1


In [2]:
# Classe noeud 
class Noeud():
    def __init__(self, indice_attribut=None, seuil=None, left=None, right=None, GI=None, valeur=None):

        self.indice_attribut = indice_attribut # L'indice de l'attribut testé dans le noeud
        self.seuil = seuil # Le seuil sur lequel on répartie les données dans le noeud
        self.left = left # Contient le noeud gauche
        self.right = right # Contient le noeud droit
        self.GI = GI # Le gain d'information
        
        # 'valeur' reste 'None' si le noeud n'est pas une feuille
        # si est une feuille, 'valeur' prend la valeur de la classe representée par la feuille
        self.valeur = valeur

class DecisionTree():
    def __init__(self, min_splits=2, profondeur_max=2):
        
        # initialise la racine de l'arbre
        self.root = None
        
        # initialisation des conditions d'arrêt
        self.min_splits = min_splits # nombres d'enfant pour chaque noeud 
        self.profondeur_max = profondeur_max
        
    #la fonction récursive d'entrainement    
    def trainingTree(self, data, profondeur_curr=0):
        
        X = data[:,:-1]
        Y = data[:,-1] 
        individus_num, attributs_num = np.shape(X)
        
        # diviser jusqu'à ce que les conditions d'arrêt soient remplies
        if individus_num>=self.min_splits and profondeur_curr<=self.profondeur_max:
            # trouver la meilleur separation qui maximise le gain
            meilleur_separation = self.meilleur_split(data, individus_num, attributs_num)
            # vérifier si le gain d'information est strictement positif
            if meilleur_separation["GI"]>0:
                # appel à trainingTree pour left
                left = self.trainingTree(meilleur_separation["data_left"], profondeur_curr+1)
                # appel à trainingTree pour right
                right = self.trainingTree(meilleur_separation["data_right"], profondeur_curr+1)
                # return le noeud et ajout les noeuds enfants et le gain
                return Noeud(meilleur_separation["indice_attribut"], meilleur_separation["seuil"], 
                            left, right, meilleur_separation["GI"])
               
        # calculer la valeur de la feuille 
        class_feuille = self.class_feuille(Y)
        # return noeud feuille 
        return Noeud(valeur=class_feuille)
    
    def meilleur_split(self, data, individus_num, attributs_num):
        
        # dictionnaire pour stocker le meilleur split
        meilleur_separation = {}
        max_GI = -1 # initialisation du max gain information
        
        # boucler sur toutes les attributs
        for indice_attribut in range(attributs_num):
            feature_valeur = data[:, indice_attribut] 
            possible_seuils = np.unique(feature_valeur) 
            # boucle sur toutes les valeurs de l'attribut présentes dans les données
            for seuil in possible_seuils:
                # Séparer les données selon le seuil
                data_left, data_right = self.split(data, indice_attribut, seuil)
                # vérifier si les enfants ne sont pas nuls
                if len(data_left)>0 and len(data_right)>0:
                    y = data[:, -1]
                    left_y = data_left[:, -1]
                    right_y = data_right[:, -1]
                    # calculer le gain d'information pour le seuil
                    GI_curr = self.information_gain(y, left_y, right_y)
                    # mettre à jour la meilleure split si nécessaire
                    if GI_curr > max_GI:
                        meilleur_separation["indice_attribut"] = indice_attribut
                        meilleur_separation["seuil"] = seuil
                        meilleur_separation["data_left"] = data_left
                        meilleur_separation["data_right"] = data_right
                        meilleur_separation["GI"] = GI_curr
                        max_GI = GI_curr
                        
        # return best split
        return meilleur_separation
    
    def split(self, data, indice_attribut, seuil): # Fonction qui sépare les données selon le seuil
        
        data_left = np.array([row for row in data if row[indice_attribut]<=seuil])
        data_right = np.array([row for row in data if row[indice_attribut]>seuil])
        return data_left, data_right
    
    def information_gain(self, parent, l_child, r_child):
        
        p_l = len(l_child) / len(parent)
        p_r = len(r_child) / len(parent)
        gain = self.entropy(parent) - (p_l*self.entropy(l_child) + p_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        classes = np.unique(y) # classes=[0,1]
        entropy = 0
        for c in classes:
            p = len(y[y == c]) / len(y)
            entropy += -p * np.log2(p)
        return entropy
    
        
    def class_feuille(self, Y): # retourne la classe qui a le max d'occurence dans Y
        Y = list(Y)
        return max(Y, key=Y.count)
    

    def print_tree(self, tree=None, indent=" "): # Afficher l'arbre
        if not tree: # initialiser tree par le noeud root
            tree = self.root
        if tree.valeur is not None: # verifier si tree est feuille -> on affiche la classe
            print(tree.valeur)
        else: # sinon -> tree est noeud
            print('---- '+data.columns[tree.indice_attribut]+' ----', "<=", tree.seuil, "?", tree.GI)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent) # afficher left
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent) # afficher left
    
    def fit(self, X, Y): # Fonction qui entraine le model 
        data = np.concatenate((X, Y), axis=1)
        self.root = self.trainingTree(data)
    
    def predict(self, X): # Fonction qui predict la class de tous les individus x dans X
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree): # Fonction qui predict la class d'un seul individus x 
        if tree.valeur!=None: return tree.valeur # Si tree est une feuille
        feature_val = x[tree.indice_attribut]
        if feature_val<=tree.seuil:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [3]:
# Separer les données d'entrainement et de test
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [4]:
# Entrainer et afficher l'arbre
Arbre = DecisionTree(min_splits=2, profondeur_max=4)
Arbre.fit(X_train,Y_train)
Arbre.print_tree()

---- age ---- <= 32 ? 0.0486715748396811
 left:---- purchase_value ---- <= 29 ? 0.09145695923770147
  left:0
  right:---- purchase_value ---- <= 48 ? 0.09853899217863382
    left:---- sex ---- <= F ? 0.4040097573248599
        left:---- source ---- <= Ads ? 0.31127812445913283
                left:0
                right:1
        right:1
    right:---- purchase_value ---- <= 203 ? 0.24014238243251246
        left:0
        right:---- age ---- <= 29 ? 0.9709505944546686
                left:0
                right:1
 right:---- source ---- <= Ads ? 0.03666866736525018
  left:0
  right:---- age ---- <= 37 ? 0.06774509098753351
    left:0
    right:---- age ---- <= 39 ? 0.29984283988623867
        left:---- purchase_value ---- <= 19 ? 0.31127812445913283
                left:1
                right:0
        right:0


In [5]:
# Calcul de la précision du modél avec la fonction accuracy_score de sklearn
Y_pred = Arbre.predict(X_test) 
print("La pécision entre Y_test et Y_pred :", accuracy_score(Y_test, Y_pred))

La pécision entre Y_test et Y_pred : 0.7391304347826086


In [6]:
from PIL import Image

# Open the image file
image = Image.open("arbre.png")

# Display the image
image.show()

In [10]:
# import graphviz

# # Création de l'objet Graph
# dot = graphviz.Graph()

# # Ajout des nœuds de l'arbre
# dot.node("Sex", "Age")
# dot.node("Class 0", "Class 0")
# dot.node("purchase_value", "purchase_value")
# dot.node("Class 1", "Class 1")
# dot.node("purchase_value_15", "purchase_value_15")

# # Ajout des arêtes de l'arbre
# dot.edge("Sex", "purchase_value", label="M")
# dot.edge("Sex", "Class 0", label="F")
# dot.edge("purchase_value", "Class 1", label="<")
# dot.edge("purchase_value", "purchase_value_15", label=">")


# # Génération de l'image de l'arbre
# dot.render("arbre.gv", view=True)

In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt

# # Création d'un graphe vide
# G = nx.Graph()

# # Ajout des nœuds de l'arbre
# G.add_node("Sex", pos=(0, 0))
# G.add_node("Class 0", pos=(-1, -1))
# G.add_node("purchase_value", pos=(1, -1))
# G.add_node("Class 1", pos=(-2, -2))
# G.add_node("purchase_value_15", pos=(2, -2))

# # Ajout des arêtes de l'arbre
# G.add_edge("Sex", "purchase_value", label="M")
# G.add_edge("Sex", "Class 0", label="F")
# G.add_edge("purchase_value", "Class 1", label="<")
# G.add_edge("purchase_value", "purchase_value_15", label=">")

# # Définition des positions des nœuds de l'arbre
# pos = nx.get_node_attributes(G, "pos")

# # Dessin des nœuds de l'arbre
# nx.draw(G, pos, with_labels=True)

# # Dessin des labels des arêtes de l'arbre
# edge_labels = nx.get_edge_attributes(G, "label")
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# # Affichage de l'arbre
# plt.show()